In [48]:
# Importing necessary libraries and data

import pandas as pd

df = pd.read_csv("C:/Users/Sandeep/Downloads/archive (1)/user_behavior_dataset.csv")

In [2]:
# Installing duckdb

%pip install duckdb


Note: you may need to restart the kernel to use updated packages.


In [50]:
# Restart and Import duckdb

import duckdb


In [52]:
# Test Query for duckdb

duckdb.query("""
    SELECT 'DuckDB is working 🚀' AS status
""").show()

┌──────────────────────┐
│        status        │
│       varchar        │
├──────────────────────┤
│ DuckDB is working 🚀 │
└──────────────────────┘



In [56]:
# Data Check

duckdb.sql("SELECT COUNT(*) AS total_users FROM df").df()


,total_users
0,1000


In [26]:
# Overall Conversion Rate (KPI)

duckdb.sql("""
SELECT
    AVG(target) AS conversion_rate
FROM df
""").df()


,conversion_rate
0,0.29


### Overall Conversion Rate

The SQL-based calculation confirms an overall conversion rate of **~29%**, matching the Python EDA results.  
This validates the consistency of conversion metrics across analytical tools and establishes a reliable baseline KPI.


In [28]:
#Engagement Metrics by Conversion

duckdb.sql("""
SELECT
    target,
    AVG(visits)     AS avg_visits,
    AVG(clicks)     AS avg_clicks,
    AVG(time_spent) AS avg_time_spent
FROM df
GROUP BY target
""").df()


,target,avg_visits,avg_clicks,avg_time_spent
0,0,9.170423,5.429577,366.525352
1,1,9.465517,5.882759,383.348276


### Engagement Metrics by Conversion

Average engagement metrics show that converted users generate higher interaction levels, particularly in **clicks** and **time_spent**, while visit frequency remains similar across both groups.  
This reinforces that conversion is driven by **engagement depth rather than visit count**.


In [18]:
# Median Engagement by C

duckdb.sql("""
SELECT
    target,
    MEDIAN(visits)     AS median_visits,
    MEDIAN(clicks)     AS median_clicks,
    MEDIAN(time_spent) AS median_time_spent
FROM df
GROUP BY target
""").df()


,target,median_visits,median_clicks,median_time_spent
0,0,9.0,5.0,348.0
1,1,9.0,6.0,369.5


### Median Engagement Comparison

Median-based SQL analysis confirms earlier findings:
- Median visits remain the same across both groups
- Converted users show higher median clicks
- Converted users spend more time on the platform

Using medians provides a robust comparison that is resistant to skewed engagement behavior.


In [30]:
#Engagement Segmentation (Clicks)

duckdb.sql("""
SELECT
    CASE 
        WHEN clicks >= 6 THEN 'High Engagement'
        ELSE 'Low Engagement'
    END AS engagement_segment,
    COUNT(*) AS users,
    AVG(target) AS conversion_rate
FROM df
GROUP BY engagement_segment
""").df()


,engagement_segment,users,conversion_rate
0,High Engagement,466,0.326180
1,Low Engagement,534,0.258427


### Engagement Segmentation Based on Clicks

Users with **high engagement (≥ 6 clicks)** show a noticeably higher conversion rate compared to low-engagement users.  
This segmentation highlights clicks as a strong behavioral threshold for identifying conversion-ready users.


In [34]:
# Session Duration Segmentation

duckdb.sql("""
SELECT
    CASE 
        WHEN time_spent >= 370 THEN 'Long Session'
        ELSE 'Short Session'
    END AS session_group,
    COUNT(*) AS users,
    AVG(target) AS conversion_rate
FROM df
GROUP BY session_group
""").df()


,session_group,users,conversion_rate
0,Long Session,457,0.317287
1,Short Session,543,0.267035


### Session Duration Impact

Users with session durations above **~370 seconds** convert at a higher rate than those with shorter sessions.  
This indicates that sustained interaction with the platform increases purchase likelihood.


In [32]:
# Combined Engagement Tier (Strong Signal)

duckdb.sql("""
SELECT
    CASE
        WHEN clicks >= 6 AND time_spent >= 370 THEN 'High Engagement'
        ELSE 'Low Engagement'
    END AS engagement_tier,
    COUNT(*) AS users,
    AVG(target) AS conversion_rate
FROM df
GROUP BY engagement_tier
""").df()


,engagement_tier,users,conversion_rate
0,High Engagement,367,0.329700
1,Low Engagement,633,0.266983


### Combined Engagement Tier Analysis

Users exhibiting both **high clicks and longer session duration** demonstrate the highest conversion rates.  
This compound segmentation confirms that conversion is driven by **multiple engagement signals working together**, rather than any single metric in isolation.


## SQL Analysis Summary

- SQL results align closely with Python-based EDA findings  
- Visit frequency alone does not meaningfully impact conversion  
- Click intensity and session duration are the strongest conversion drivers  
- Engagement-based segmentation reveals clear conversion lift  
- SQL provides a scalable, production-style validation of analytical insights
